In [2]:
import os
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split # Functipn to split data into training, validation and test sets

from tqdm import tqdm 
from tqdm.notebook import tqdm_notebook
import datetime
import time


# Torch
import torch
from torchvision import transforms
import torchvision.models as models
import torch.nn as nn
import torchinfo


In [3]:
moas = ["cyclooxygenase inhibitor", "dopamine receptor antagonist","adrenergic receptor antagonist", "phosphodiesterase inhibitor",  "HDAC inhibitor", 
             "histamine receptor antagonist","EGFR inhibitor", "adrenergic receptor agonist", "PARP inhibitor",  "topoisomerase inhibitor"]


In [28]:
clue_sig_in_SPECS = pd.read_csv('/home/jovyan/Tomics-CP-Chem-MoA/04_Tomics_Models/init_data_expl/clue_sig_in_SPECS1&2.csv', delimiter = ",")
clue_sig_in_SPECS.columns

Index(['Unnamed: 0', 'pert_id', 'Compound ID', 'Batch nr', 'moa', 'bead_batch',
       'nearest_dose', 'pert_dose', 'pert_dose_unit', 'pert_idose',
       'pert_itime', 'pert_time', 'pert_time_unit', 'cell_mfc_name',
       'pert_mfc_id', 'nsample', 'cc_q75', 'ss_ngene', 'tas',
       'pct_self_rank_q25', 'wt', 'median_recall_rank_spearman',
       'median_recall_rank_wtcs_50', 'median_recall_score_spearman',
       'median_recall_score_wtcs_50', 'batch_effect_tstat',
       'batch_effect_tstat_pct', 'is_hiq', 'qc_pass', 'sig_id', 'pert_type',
       'cell_iname', 'det_wells', 'det_plates', 'distil_ids', 'build_name',
       'project_code', 'cmap_name', 'is_exemplar_sig', 'is_ncs_sig',
       'is_null_sig'],
      dtype='object')

In [20]:
moas = ["cyclooxygenase inhibitor", "adrenergic receptor antagonist"]

In [26]:
clue_sig_in_SPECS= clue_sig_in_SPECS[clue_sig_in_SPECS["cc_q75"] > 0.1]

In [36]:
MoAs_2_correlated = {"cyclooxygenase inhibitor": ["A375", "HA1E", "HCC515", "PC3", "VCAP", "HEPG2", "HT29", "MCF7", "A549"], 
 "adrenergic receptor antagonist" : ["A375", "HA1E", "HCC515", "VCAP", "HT29", "MCF7", "A549"] }

together = []
for i in moas:
    bro = MoAs_2_correlated[i]
    svt = clue_sig_in_SPECS[clue_sig_in_SPECS["moa"]== i]
    yep = svt[svt["cell_iname"].isin(bro)]
    together.append(yep)
allbo = pd.concat(together)

In [39]:
allbo.shape[0]*0.2

238.60000000000002

In [40]:
159+ 15+ 50 + 22

246

In [27]:
clue_sig_in_SPECS[clue_sig_in_SPECS["moa"].isin(moas)]



,Unnamed: 0,pert_id,Compound ID,Batch nr,moa,bead_batch,nearest_dose,pert_dose,pert_dose_unit,pert_idose,...,cell_iname,det_wells,det_plates,distil_ids,build_name,project_code,cmap_name,is_exemplar_sig,is_ncs_sig,is_null_sig
1291,1291,K12994359,CBK290981,BJ1898795,cyclooxygenase inhibitor,b33,3.33,3.33333,uM,3.33 uM,...,PHH,D11,LRS001_PHH.MGH_24H_X1_B33|LRS001_PHH.MGH_24H_X...,LRS001_PHH.MGH_24H_X1_B33:D11|LRS001_PHH.MGH_2...,NaN,LRS,valdecoxib,0.0,1.0,0.0
1292,1292,K12994359,CBK290981,BJ1898795,cyclooxygenase inhibitor,b33,1.11,1.11111,uM,1.11 uM,...,PHH,D12,LRS001_PHH.MGH_24H_X1_B33|LRS001_PHH.MGH_24H_X...,LRS001_PHH.MGH_24H_X1_B33:D12|LRS001_PHH.MGH_2...,NaN,LRS,valdecoxib,0.0,1.0,0.0
1293,1293,K12994359,CBK290981,BJ1898795,cyclooxygenase inhibitor,b33,10.00,10.00000,uM,10 uM,...,PHH,D10,LRS001_PHH.MGH_24H_X1_B33|LRS001_PHH.MGH_24H_X...,LRS001_PHH.MGH_24H_X1_B33:D10|LRS001_PHH.MGH_2...,NaN,LRS,valdecoxib,1.0,1.0,0.0
1294,1294,K12994359,CBK290981,BJ1898795,cyclooxygenase inhibitor,b5,200.00,177.60000,uM,200 uM,...,RMUGS,H02,CPC006_RMUGS_6H_X1_B5_DUO52HI53LO|CPC006_RMUGS...,CPC006_RMUGS_6H_X1_B5_DUO52HI53LO:H02|CPC006_R...,NaN,CPC,valdecoxib,0.0,0.0,0.0
1295,1295,K12994359,CBK290981,BJ1898795,cyclooxygenase inhibitor,f2b4,200.00,177.60000,uM,200 uM,...,SW620,H02,CPC006_SW620_6H_X1_F2B4_DUO52HI53LO|CPC006_SW6...,CPC006_SW620_6H_X1_F2B4_DUO52HI53LO:H02|CPC006...,NaN,CPC,valdecoxib,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52305,52305,A43974499,CBK201001H,BJ1895720,adrenergic receptor antagonist,b3,10.00,10.00000,uM,10 uM,...,HA1E,L21,CPC001_HA1E_6H_X1_B3_DUO52HI53LO|CPC001_HA1E_6...,CPC001_HA1E_6H_X1_B3_DUO52HI53LO:L21|CPC001_HA...,NaN,CPC,reboxetine,0.0,1.0,0.0
52432,52432,A47884604,CBK041335,BJ1894255,adrenergic receptor antagonist,b6,10.00,10.00000,uM,10 uM,...,NPC,K19,NMH001_NPC_6H_X1_B6_DUO52HI53LO|NMH001_NPC_6H_...,NMH001_NPC_6H_X1_B6_DUO52HI53LO:K19|NMH001_NPC...,NaN,NMH,cyanopindolol,0.0,1.0,0.0
52433,52433,A47884604,CBK041335,BJ1894255,adrenergic receptor antagonist,b6,10.00,10.00000,uM,10 uM,...,NEU,K19,NMH001_NEU_24H_X1.L2_B6_DUO52HI53LO|NMH001_NEU...,NMH001_NEU_24H_X1.L2_B6_DUO52HI53LO:K19|NMH001...,NaN,NMH,cyanopindolol,1.0,1.0,0.0
52435,52435,A47884604,CBK041335,BJ1894255,adrenergic receptor antagonist,b6,10.00,10.00000,uM,10 uM,...,NPC,K19,NMH001_NPC_24H_X1_B6_DUO52HI53LO|NMH001_NPC_24...,NMH001_NPC_24H_X1_B6_DUO52HI53LO:K19|NMH001_NP...,NaN,NMH,cyanopindolol,0.0,1.0,0.0


In [125]:
# identifying rows with only landmark genes
clue_gene = pd.read_csv('/home/jovyan/Tomics-CP-Chem-MoA/04_Tomics_Models/init_data_expl/clue_geneinfo_beta.txt', delimiter = "\t")

# Pre-processing

## Psuedo-Code
1. Do pre-processing to extract relevant transcriptomic profiles with MoAs of interest from the GCTX document
2. Prepare classes if necessary, by for example do one-hot encoding
3. Do the test, train  and validation split, making sure to shuffle
4. Save the test, train and validation splits to a csv.

In [126]:
clue_sig_in_SPECS.shape[0]

53975

In [127]:
profile_ids = clue_sig_in_SPECS[["Compound ID", "sig_id", "moa"]][clue_sig_in_SPECS["moa"].isin(moas)]

In [128]:
profile_ids.shape[0]

18124

In [129]:
profile_ids

,Compound ID,sig_id,moa
1168,CBK041182,LPROT005_MCF10A_6H:BRD-K37798499:0.1,topoisomerase inhibitor
1169,CBK041182,LPROT004_YAPC_6H:BRD-K37798499:5,topoisomerase inhibitor
1170,CBK041182,LPROT002_A375_6H:BRD-K37798499:5,topoisomerase inhibitor
1171,CBK041182,LPROT003_PC3_6H:BRD-K37798499:5,topoisomerase inhibitor
1172,CBK041182,LPROT005_MCF10A_6H:BRD-K37798499:1,topoisomerase inhibitor
...,...,...,...
53771,CBK200956,CPC005_A375_24H:BRD-A60197193-001-05-4:10,dopamine receptor antagonist
53772,CBK200956,CPC005_HA1E_24H:BRD-A60197193-001-05-4:10,dopamine receptor antagonist
53773,CBK200956,CPC005_HT29_6H:BRD-A60197193-001-05-4:10,dopamine receptor antagonist
53774,CBK200956,CPC011_PC3_24H:BRD-A60197193-001-09-6:10,dopamine receptor antagonist


In [130]:
dictionary = {"cyclooxygenase inhibitor": 9, "dopamine receptor antagonist": 8,"adrenergic receptor antagonist": 7, "phosphodiesterase inhibitor": 6,  "HDAC inhibitor" : 5, 
             "histamine receptor antagonist": 4,"EGFR inhibitor": 3, "adrenergic receptor agonist": 2, "PARP inhibitor":
             1,  "topoisomerase inhibitor": 0}

In [131]:
num_classes = len(dictionary) 

for i in range(profile_ids.shape[0]):
    profile_ids.iloc[i, 2] = dictionary[profile_ids.iloc[i, 2]]# change moa to classes 


In [132]:
profile_ids

,Compound ID,sig_id,moa
1168,CBK041182,LPROT005_MCF10A_6H:BRD-K37798499:0.1,0
1169,CBK041182,LPROT004_YAPC_6H:BRD-K37798499:5,0
1170,CBK041182,LPROT002_A375_6H:BRD-K37798499:5,0
1171,CBK041182,LPROT003_PC3_6H:BRD-K37798499:5,0
1172,CBK041182,LPROT005_MCF10A_6H:BRD-K37798499:1,0
...,...,...,...
53771,CBK200956,CPC005_A375_24H:BRD-A60197193-001-05-4:10,8
53772,CBK200956,CPC005_HA1E_24H:BRD-A60197193-001-05-4:10,8
53773,CBK200956,CPC005_HT29_6H:BRD-A60197193-001-05-4:10,8
53774,CBK200956,CPC011_PC3_24H:BRD-A60197193-001-09-6:10,8


## Train and Test Set Splitting
Pseudocode
1. extract all of the compounds from that have transcriptomic profiles
(could probably add check to see how many transcriptomic profiles a compound has)
2. split the compounds into a train, test and validation data set
3. create list of compound names for each set

In [133]:
compound_split = profile_ids.drop_duplicates(subset=["Compound ID"])

In [134]:
compound_split["moa"]

1168     0
1291     9
1363     7
2155     9
2225     7
        ..
52412    2
52431    7
53004    8
53745    2
53746    8
Name: moa, Length: 241, dtype: object

In [177]:
# --------------------------- splitting into training, validation and test sets----------#
# split dataset into test and training/validation sets (10-90 split)
compound_train_valid, compound_test, compound_train_valid_moa, test_Y = train_test_split(
compound_split, compound_split["moa"],  stratify=compound_split["moa"], 
    shuffle = True, test_size = 0.15, random_state = 1)


In [178]:
print(int(compound_train_valid.shape[0]))

print(int(test_y.shape[0]))

204
37


In [179]:
test_y

,Compound ID,sig_id,moa
33172,CBK277909,CPC011_HT29_6H:BRD-K69650333-003-11-6:10,0
37216,CBK041882,LIVB001_HUH7_72H:E08,4
35289,CBK307870,CPC004_HCC515_6H:BRD-K68558722-001-03-2:10,9
10045,CBK012001,CPD002_PC3_24H:BRD-K38323065-001-11-6:10,9
43345,CBK040873,LIVB001_HUH7_72H:K03,2
46935,CBK042049,NMH002_NEU_24H:BRD-K98530306-003-13-4:10,2
10199,CBK041407,CPD003_PC3_24H:BRD-K82562631-001-01-0:10,9
2916,CBK011699,NMH002_NPC_24H:BRD-K01292756-001-20-1:10,8
13937,CBK288256,LUNG001_A549_24H:BRD-K72420232:10,3
9932,CBK011672,CVD001_HEPG2_6H:BRD-K25394294-001-04-0:10,9


In [180]:
assert (int(compound_train_valid.shape[0]) + int(compound_test.shape[0])) == int(compound_split.shape[0])

In [182]:


# Split data set into training and validation sets (1 to 9)
# Same as above, but does split of only training data into training and validation 
# data (in order to take advantage of stratification parameter)
compound_train, compound_valid, moa_train, moa_valid = train_test_split(
compound_train_valid, compound_train_valid["moa"], test_size = 0.15, shuffle= True,
    stratify = compound_train_valid["moa"],
      random_state = 62757)


In [183]:
cmpd_trai_lst = list(compound_train["Compound ID"])
cmpd_vali_lst = list(compound_valid["Compound ID"])
cmpd_tes_lst = list(compound_test["Compound ID"])


In [184]:
assert (int(compound_train.shape[0]) + int(compound_valid.shape[0])) == int(compound_train_valid.shape[0])

In [185]:
def train_test_valid_split(train_cpds, valid_cpds, test_cpds, df):
    '''
    Splitting the original data set into train and test at the compound level'''
    df_train = df.loc[df["Compound ID"].isin(train_cpds)]
    df_valid = df.loc[df["Compound ID"].isin(valid_cpds)]
    df_test = df.loc[df["Compound ID"].isin(test_cpds)]
    return df_train, df_valid, df_test

In [186]:
compound_train

,Compound ID,sig_id,moa
18527,CBK289739,NMH002_NPC_24H:BRD-K67102207-236-03-6:10,5
14291,CBK293871,ERAM001_A549_24H:BRD-K33610132:3.95,3
31818,CBK041678,CPD003_PC3_24H:BRD-K78113049-001-10-5:10,0
48343,CBK013140,CPD002_MCF7_24H:BRD-A02990301-001-01-1:10,2
41735,CBK200900,CPC001_HA1E_6H:BRD-K16277217-001-03-4:10,8
...,...,...,...
10125,CBK016300,CPD001_MCF7_6H:B08,9
18545,CBK289740,NMH002_NPC_24H:BRD-K45528773-001-04-9:10,5
50978,CBK041399,CPD002_MCF7_24H:BRD-A28856712-003-19-5:10,2
49914,CBK200730,LIVB002_HUH7_72H:H22,8


In [187]:
profile_ids[profile_ids["Compound ID"].isin(cmpd_trai_lst)]

,Compound ID,sig_id,moa
1168,CBK041182,LPROT005_MCF10A_6H:BRD-K37798499:0.1,0
1169,CBK041182,LPROT004_YAPC_6H:BRD-K37798499:5,0
1170,CBK041182,LPROT002_A375_6H:BRD-K37798499:5,0
1171,CBK041182,LPROT003_PC3_6H:BRD-K37798499:5,0
1172,CBK041182,LPROT005_MCF10A_6H:BRD-K37798499:1,0
...,...,...,...
53771,CBK200956,CPC005_A375_24H:BRD-A60197193-001-05-4:10,8
53772,CBK200956,CPC005_HA1E_24H:BRD-A60197193-001-05-4:10,8
53773,CBK200956,CPC005_HT29_6H:BRD-A60197193-001-05-4:10,8
53774,CBK200956,CPC011_PC3_24H:BRD-A60197193-001-09-6:10,8


In [188]:
training_set, validation_set, test_set = train_test_valid_split(cmpd_trai_lst, cmpd_vali_lst, cmpd_tes_lst, profile_ids)

In [189]:
training_set.head()

,Compound ID,sig_id,moa
1168,CBK041182,LPROT005_MCF10A_6H:BRD-K37798499:0.1,0
1169,CBK041182,LPROT004_YAPC_6H:BRD-K37798499:5,0
1170,CBK041182,LPROT002_A375_6H:BRD-K37798499:5,0
1171,CBK041182,LPROT003_PC3_6H:BRD-K37798499:5,0
1172,CBK041182,LPROT005_MCF10A_6H:BRD-K37798499:1,0


In [190]:
profile_ids.head()

,Compound ID,sig_id,moa
1168,CBK041182,LPROT005_MCF10A_6H:BRD-K37798499:0.1,0
1169,CBK041182,LPROT004_YAPC_6H:BRD-K37798499:5,0
1170,CBK041182,LPROT002_A375_6H:BRD-K37798499:5,0
1171,CBK041182,LPROT003_PC3_6H:BRD-K37798499:5,0
1172,CBK041182,LPROT005_MCF10A_6H:BRD-K37798499:1,0


# Saving Split Data to Subdirectory

In [191]:
save_direct = "/home/jovyan/Tomics-CP-Chem-MoA/04_Tomics_Models/data_split_csvs/"



In [198]:
def save_to_csv(df, dir_path, file_name, compress = None):
    '''Saving train, test or valid set to specific directory with or without compression'''
    if not os.path.exists(dir_path):
        print("Making path")
        os.mkdir(dir_path)
    df.to_csv(dir_path + file_name + ".csv", index = False, compression = compress)

In [199]:
save_to_csv(training_set, save_direct, "L1000_training_set")
save_to_csv(validation_set, save_direct, "L1000_valid_set")
save_to_csv(test_set, save_direct, "L1000_test_set")

In [8]:
testing = pd.read_csv('/home/jovyan/Tomics-CP-Chem-MoA/04_Tomics_Models/data_split_csvs/L1000_training_set_cyclo_adr_2_cell.csv', delimiter= ',')

In [9]:
testing.cell_iname.unique()

array(['PC3', 'MCF7', 'A549', 'A375', 'VCAP', 'HEPG2', 'HCC515', 'HT29',
       'HA1E'], dtype=object)

In [10]:
testing.shape

(947, 4)

In [11]:
testing.moa.value_counts()

0    685
1    262
Name: moa, dtype: int64

In [21]:
valid1 = pd.read_csv('/home/jovyan/Tomics-CP-Chem-MoA/04_Tomics_Models/data_split_csvs/L1000_test_set_cyclo_adr_2_cell.csv', delimiter= ',')

In [23]:
valid1.moa.value_counts()

0    174
1     72
Name: moa, dtype: int64

In [13]:
testing2 = pd.read_csv('/home/jovyan/Tomics-CP-Chem-MoA/04_Tomics_Models/data_split_csvs/L1000_training_set_cyclo_adr_2_cell_APC2.csv', delimiter= ',')

In [14]:
testing2

,Compound ID,sig_id,moa,cell_iname
0,CBK041670,CPC016_A549_6H:BRD-A22844106-001-16-1:10,0,A549
1,CBK041670,CPC016_MCF7_6H:BRD-A22844106-001-16-1:10,0,MCF7
2,CBK041670,CPC004_HEPG2_6H:BRD-A22844106-001-16-1:10,0,HEPG2
3,CBK041670,CPC016_HEPG2_6H:BRD-A22844106-001-16-1:10,0,HEPG2
4,CBK041670,CPC004_HCC515_6H:BRD-A22844106-001-16-1:10,0,HCC515
...,...,...,...,...
282,CBK201001H,CPC001_HCC515_24H:BRD-A43974499-066-01-1:10,1,HCC515
283,CBK201001H,CPC001_HCC515_6H:BRD-A43974499-066-01-1:10,1,HCC515
284,CBK201001H,CPC017_HT29_6H:BRD-A43974499-066-01-1:10,1,HT29
285,CBK201001H,CPC017_A549_6H:BRD-A43974499-066-01-1:10,1,A549


In [15]:
testing2.moa.value_counts()

0    200
1     87
Name: moa, dtype: int64

In [18]:
valid2 = pd.read_csv('/home/jovyan/Tomics-CP-Chem-MoA/04_Tomics_Models/data_split_csvs/L1000_test_set_cyclo_adr_2_cell_APC2.csv', delimiter= ',')

In [19]:
valid2

,Compound ID,sig_id,moa,cell_iname
0,CBK290981,CPC006_A549_24H:BRD-K12994359-001-07-7:177.6,0,A549
1,CBK290981,CPC006_MCF7_6H:BRD-K12994359-001-07-7:177.6,0,MCF7
2,CBK290981,CPC006_MCF7_24H:H02,0,MCF7
3,CBK290981,CPC006_A375_24H:BRD-K12994359-001-07-7:177.6,0,A375
4,CBK290981,CPC006_HA1E_24H:H02,0,HA1E
...,...,...,...,...
74,CBK041695,CPC004_HT29_6H:BRD-A07440155-003-16-3:10,1,HT29
75,CBK041695,CPC020_A549_6H:BRD-A07440155-003-22-1:10,1,A549
76,CBK041695,CPC020_VCAP_24H:BRD-A07440155-003-22-1:10,1,VCAP
77,CBK041464,CPC020_VCAP_6H:BRD-A29260609-003-21-1:10,1,VCAP
